# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Action Points

### 1. No need to make 10 transactions a day. Just one is enough
### 2. Close diff (Movement of close value over a day) looks promising to find trend or ranging market

data                = get_date_list(data)
data                = read_data(data)

In [2]:
%%time
data = run_engine(data)

100%|██████████████████████████████████| 74387/74387 [00:18<00:00, 3998.76it/s]


pl    0.12863
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
65420 2020-11-18 04:10:00         NaN  sema_close  0.00513
65943 2020-11-19 23:45:00         NaN  sema_close -0.00328
66181 2020-11-20 19:35:00         NaN  sema_close -0.00187
66500 2020-11-23 22:10:00         NaN  sema_close -0.00595
67302 2020-11-26 17:00:00         NaN  sema_close  0.00227
67951 2020-11-30 23:05:00         NaN  sema_close -0.00077
69106 2020-12-06 23:20:00         NaN  sema_close  0.00821
69814 2020-12-09 10:20:00         NaN  sema_close -0.00110
70472 2020-12-11 17:10:00         NaN  sema_close -0.00162
71905 2020-12-18 16:35:00         NaN  sema_close  0.00657
72281 2020-12-21 23:55:00         NaN  sema_close -0.00069
72482 2020-12-22 16:40:00         NaN  sema_close -0.00781
72758 2020-12-23 15:40:00         NaN  sema_close -0.00350
73087 2020-12-24 19:05:00         NaN  sema_close -0.00286
74305 2020-12-31 15:10:00         NaN  sema_clo

100%|██████████████████████████████████████| 223/223 [00:00<00:00, 1242.95it/s]

Wall time: 1min 45s


In [3]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1     0.00381
2     0.01905
3     0.07279
4    -0.00276
5     0.01922
6     0.01887
7     0.00787
8    -0.01320
9     0.01088
10   -0.01633
11    0.00694
12    0.00149
Name: pl, dtype: float64
---------------------
max : 3 : 0.07279
min : 10 : -0.016329999999999997
---------------------
monthly_avg : 0.0107


In [4]:
data['lema_gap_pl_analysis_month'] = 8

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
3,0.000,0.0,1.0,-1.0,0.00000,-0.00343,-0.00343
2,0.001,2.0,1.0,1.0,0.00551,-0.00441,0.00110
0,0.002,1.0,3.0,-2.0,0.00183,-0.00658,-0.00475
6,0.003,1.0,0.0,1.0,0.00279,0.00000,0.00279
1,0.004,0.0,1.0,-1.0,0.00000,-0.00538,-0.00538
5,0.005,1.0,0.0,1.0,0.00045,0.00000,0.00045
4,0.006,0.0,1.0,-1.0,0.00000,-0.00195,-0.00195


In [5]:
sys.exit()

SystemExit: 

In [ ]:
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl_type'].count().unstack())
print('----------------------------------------------')
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl'].sum().unstack())

In [ ]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [244]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
month = 8

# Monthly Chart -------------------
data['plot_start']  = f"{data['input_year']}-{month}-1"
data['plot_stop']   = f"{data['input_year']}-{month+1}-1"
# Monthly Chart -------------------

# December Chart ----------------------------
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# December Chart ----------------------------

# Year Chart ----------------------------
# data['plot_start']  = f"{data['input_year']}-1-1"
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# Year Chart ----------------------------

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
# data['things_to_plot'] = ['candles','sema', 'slema','positions','lema', 'lema_angle', 'lema_max_min', 'close_diff']
# data['things_to_plot'] = ['lema', 'tip', 'candles', 'lema_angle', 'lema_change']
data['things_to_plot'] = ['lema', 'candles', 'lema_angle', 'lema_change']

plot_graph(data)

generate_result_report(data)

In [242]:
def get_lema_pick_points(data)
    a = np.array(data['df_ohlc']['lema'].diff())

    a1 = np.sign(a)

    for i in tqdm(np.arange(1, len(a1))):
        if a1[i] == 0:        
            a1[i] = a1[i+1]

        if a1[i] != a1[i-1]:
            if a1[i+1] == a1[i-1]:
                a1[i] = a1[i-1]

    idx2 = []

    for i in np.arange(len(a1)-1):
        if i > 2:
            if a1[i-1] == a1[i-2]:
                if a1[i] == a1[i+1]:
                    if a1[i] != a1[i-1]:
                        idx2.append(i)

    idx2 = (np.array(idx2))
    return(data)

100%|████████████████████████████████| 74386/74386 [00:00<00:00, 765403.24it/s]


In [243]:
data['df_ohlc']['lema_change'] = np.nan
data['df_ohlc'].loc[idx2, 'lema_change'] = data['df_ohlc']['lema']

In [239]:
data['df_ohlc'].loc[(data['df_ohlc']['month_val'] == 8)
                & (data['df_ohlc']['date_val'] == 6)
                & (data['df_ohlc']['hour_val'] == 13), ['DateTime_frmt', 'lema']]

,DateTime_frmt,lema
44230,2020-08-06 13:00:00,1.185984
44231,2020-08-06 13:05:00,1.185970
44232,2020-08-06 13:10:00,1.185956
44233,2020-08-06 13:15:00,1.185931
44234,2020-08-06 13:20:00,1.185913
44235,2020-08-06 13:25:00,1.185898
44236,2020-08-06 13:30:00,1.185883
44237,2020-08-06 13:35:00,1.185874
44238,2020-08-06 13:40:00,1.185869
44239,2020-08-06 13:45:00,1.185872


In [240]:
data['df_ohlc'].loc[(data['df_ohlc']['month_val'] == 8)
                & (data['df_ohlc']['date_val'] == 6)
                & (data['df_ohlc']['hour_val'] == 14), ['DateTime_frmt', 'lema']]

,DateTime_frmt,lema
44242,2020-08-06 14:00:00,1.185861
44243,2020-08-06 14:05:00,1.185834
44244,2020-08-06 14:10:00,1.185811
44245,2020-08-06 14:15:00,1.185786
44246,2020-08-06 14:20:00,1.185762
44247,2020-08-06 14:25:00,1.185730
44248,2020-08-06 14:30:00,1.185698
44249,2020-08-06 14:35:00,1.185669
44250,2020-08-06 14:40:00,1.185637
44251,2020-08-06 14:45:00,1.185607


In [241]:
x = 44237

pd.Series(a1)[x-10:x+10]

44227   -1.0
44228   -1.0
44229   -1.0
44230   -1.0
44231   -1.0
44232   -1.0
44233   -1.0
44234   -1.0
44235   -1.0
44236   -1.0
44237   -1.0
44238   -1.0
44239    1.0
44240    1.0
44241   -1.0
44242   -1.0
44243   -1.0
44244   -1.0
44245   -1.0
44246   -1.0
dtype: float64

In [144]:
len(idx1)

1860

In [157]:
x = 1050
idx1[x:x+50]

array([41644, 41645, 42093, 42175, 42176, 42196, 42212, 42222, 42226,
       42228, 42230, 42248, 42252, 42253, 42267, 42268, 42275, 42376,
       42479, 42687, 42688, 42689, 42690, 42693, 42789, 43044, 43045,
       43048, 43414, 43425, 43426, 43432, 43506, 43513, 43517, 43520,
       43521, 43628, 44191, 44192, 44239, 44379, 44380, 44381, 44514,
       44515, 44516, 44517, 44722, 44727], dtype=int64)

In [209]:
len(idx2)

1513

In [211]:
x = 850
y = idx2[x:x+50]
for val in y:
    print(val, a[val])

40996 -6.000000000172534e-06
40999 2.0000000000575113e-06
41007 -5.000000000032756e-06
41012 1.9999999998354667e-06
41014 -1.000000000139778e-06
41016 8.000000000008e-06
41334 -2.9999999999752447e-06
41360 5.999999999950489e-06
41535 -2.0000000000575113e-06
41537 2.0000000000575113e-06
41539 -9.999999999177334e-07
41553 3.0000000001972893e-06
41569 -2.9999999999752447e-06
41571 5.000000000032756e-06
41586 -9.999999999177334e-07
41593 4.000000000115023e-06
41600 -2.9999999999752447e-06
41625 9.999999999177334e-07
42093 -9.999999999177334e-07
42187 5.000000000032756e-06
42196 -1.000000000139778e-06
42212 1.000000000139778e-06
42215 -5.999999999950489e-06
42222 2.0000000000575113e-06
42226 -2.0000000000575113e-06
42228 2.0000000000575113e-06
42230 -4.000000000115023e-06
42248 9.999999999177334e-07
42275 -4.000000000115023e-06
42376 9.999999999177334e-07
42479 -2.0000000000575113e-06
42500 6.999999999868223e-06
42658 -5.000000000032756e-06
42663 9.999999999177334e-07
42693 -4.0000000001150